### Naive Bayes

Desribes the probability of an event under certain known conditions.

**Establishment premise**: Each event feature is basically independent and has no relationship with other events.
**Feature**: Not prone to overfitting

### Theory: Building a spam filter

#### Very simple version

Use Bayes'theorem to calculate the probability of "this email is spam":
```
P(S|B)=[P(B|S)P(S)/[P(B|S)P(S)+P(B|¬S)]P(¬S)
```

* S: The event that this email is spam
* B: The event that this email contains the word bitcoin

If there are a lot of emails that are definitely spam, and a lot of emails that are definitely not spam.
The values of P(B|S) and P(¬S) can be easily estimated.

If we futher assume that the probability of an email being spam of not is half and half. (i.e. P(S)=P(¬S)=0.5)
Then the original formula can be rewritten as:
```
P(S|B)=[P(B|S)/[P(B|S)P(S)+P(B|¬S)]
```
Assumptions:
* 50% of spam emails contain the word bitcoin
* 1% of non-spam emails contain the world bitcoin -> The probability that it is indeed spam is:
```
0.5/(0.5+0.01)=98%
```

#### A more sphisticated version
**Assumption**.
* There is a vocabulary containing many words w₁,...,wɴ.
* Let Xi represent the event that the email contains the word Wi

**Assume again that we can estimate**:
* The probability that a spam email contains wi P(Xi|S)
* The probability that a spam email does not contain wi P(Xi|¬S)

```
P(X₁=X₁,...,Xn=xn|S) = P(X₁=x₁|S)×...×P(Xn=xn|S)
```

Assume that there are only two words in the vocabulary: "bitcion" and "rolex", and among all known spam emails:
* Half have "earn bitcoin"
* Half have "authenic rolex"

In this case, the probability that a spam email contains both "bitcion" and "rolex" is 0.
If we use the simple Bayesian method to estimate the probability:
```
P(X₁=1,X₂=1|S) = P(X₁=1|S)×P(X₂=1|S) = .5×.5 = .25
```
In this assumption, it is not taken into account that the two words "bitcion" and "rolex" will still affect each other.
Rather than ideal independent events. Ignoring the gap between the ideal situation and the actual situation, this model is still often used in real-world spam filters.

In practice, it is usually best to avoid multiplying many probability values.
After multiplying multiple floating-point numbers less than zero, it is easy to cause "**underflow**".
According to the algebraic relationship:
* log(ab)=log(a)+log(b)
* exp(log(x))=x

We can convert the original formula to:
```
exp(log(p1)+...+log(pn))
```
Suppose we have a large number of marked spam and non-spam emails for training.
(log(x))=x

We can convert the original formula to:
```
exp(log(p1)+...+log(pn))
```
Suppose we have a large number of marked spam and non-spam emails for training.
Then we can estimate the **probability P(Xi|S)** that the spam content contains the word wi

However, there will still be problems in the probability calculation:
Suppose that in all the emails we use for training, the word "data" in the vocabulary only appears in non-spam emails.
```
P("data"|S)=0
```
This means that the classifier will classify any email containing the word "**data**" as **non-spam**.
Even if there is a sentence like "data on free bitcoin and authetic watches".

In this case, a **pseudo-count value k** is used.
When estimating the occurrence of the word wi in spam emails, the following approach is taken:
```
P(Xi|S) = (k+number of emails containing the word wi in all spam emails)/(2k+number of all spam emails)
```
```
P(Xi|¬S) = (k+number of emails containing the word wi in all non-spam emails)/(2k+number of all non-spam emails)
```
For example:
If the word "data" does not appear in 98 spam emails, and the value of k is set to 1.


## Implementaton

### **How to create objects**
1. Split the email content by words (tokenize)
2. Collect different word tokens
3. Use re.fill extract all word tokens
4. Use **set()** to collect repeated tokens

In [10]:
from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower()                         # Convert to lowercase,
    all_words = re.findall("[a-z0-9]+", text)   # extract the words, and
    return set(all_words)                       # remove duplicate

In [11]:
assert tokenize("Data Science is science") == {"data", "science", "is"}

### Define data types for training data (basic objects)

Since the classifier needs to continously record the word tokens and various counts and labels that appear in the training data, the object classification method is adopted.

In [12]:
from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool

### Define the method to process the data type (**method**)

* ham: non-spam
* spam: spam

```python
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # smooting factor

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
```

### Define **method**: train on a bunch of emails
**method**: train on bunch of emails
1. Accumulate the number of spam and non-spam emails
2. For each word token after tokenizing each email, determine whether the email is spam to determine the comulative **token_spam_counts** or **token_ham_counts**

```python
def train(self, messages: Iterable[Message]) -> None:
    for message in messages:

        # Comulative number of emails
        if message.is_spam:
            self.spam_messages += 1
        else:
            self.ham_messages += 1

        # Comulative number of word occurences
        for token in tokenize(message.text):
            self.tokens.add(token)
            if message.is_spam:
                self.token_spam_counts[token] += 1
            else:
                self.token_ham_counts[token] += 1

```


### Definition **Method**: Predict the probability value of this event P(*spam*|*token*)
That is, the probability of the email being spam when a certain token appears.

**Goal**; Get P(token|spam) and P(token|ham) corresponding to each token in the vocabulary
**Method**: Create a private auxiliary function to perform the calculation

```python
def _probabilities(self,token:str) ->Tuple[float,float]:
"""Return P(token|spam) and P(token|ham) """
spam = self.token_spam_counts[token]
ham = self.token_ham_counts[token]

p_token_spam = (spam + self.k)/(self.spam_messages + 2 * self.k)
p_token_ham = (ham + self.k)/(self.ham_messages + 2 * self.k)

return p_token_spam,p_token_ham
```

### Definition **Method**: Writing prediction methods
In calculating the final probability value, logarithmic addition is used instead of direct multiplication.

```python
def predict(self,text:str) ->float:
text_tokens = tokenize(text)
log_prob_if_spam = log_prob_if_ham = 0.0

# Iterate over each word in the vocabulary
for token in self.tokens:
prob_if_spam, prob_if_ham = self._probabilities(token)

# If *token* appears in the email,
# add the log probability of "seeing the token"
if token in text_tokens:
log_prob_if_spam += math.log(prob_if_spam)
log_prob_if_ham += math.log(prob_if_ham)

# Otherwise add the log probability of "not seeing the token"
# The log probability of "not seeing the token" is log(1 - the probability of seeing the token)
else:
log_prob_if_spam += math.log(1.0 - log_prob_if_spam)
log_prob_if_ham += math.log(1.0 - log_prob_if_ham)

prob_if_spam = math.exp(log_prob_if_spam)
prob_if_ham = math.exp(log_prob_if_ham)
return prob_if_spam/(prob_if_spam + prob_if_ham)

```

### At this point, the classifier has been built!

In [13]:
from typing import List, Tuple, Dict, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k      # smoothing factor

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        """Predict the probability value of
        Returns P(token | spam) and P(token | ham)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        """Write prediction method"""

        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterate over each word in the vocabulary
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # If *token* appears in the email,
            # Add the log probability of "seeing the token"
            if token in text_tokens:
                log_prob_if_spam = math.log(prob_if_spam)
                log_prob_if_ham = math.log(prob_if_ham)

            # Otherwise add the log probability of "not seeing the token"
            # The log probability of "not seeing the token" is log(1 - the probability of seeing the token)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham = math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)

        return prob_if_spam / (prob_if_spam * prob_if_ham)

## Testing Our Model

In [14]:
# Write some unit tests to make sure the model works correctly
message = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5) # Assume smoothing factor is 0.5
model.train(message)

#### 1. Check whether it can calculate various couont values correctly

In [15]:
assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

### 2. Make predictions
Manually perform simple Bayesian logic calculations and confirm that the same results can be obtained:

Assume the text is: "hello spam"

* Analyze this text:
* Number of occurences of "spam": 1
* Number of occurences of "ham": 0
* Number of occurences of "rules": 0
* Number of occurences of "hello": 1

In [16]:
text = "hello spam"

probs_if_spam =[
  (1+0.5)/(1+2*0.5),  #"spam"：有這個詞
  1-(0+0.5)/(1+2*0.5), #"ham"：沒有這個詞
  1-(1+0.5)/(1+2*0.5), #"rules"：沒有這個詞
  (0+0.5)/(1+2*0.5)   #"hello"：有這個詞
]

probs_if_ham =[
  (0+0.5)/(2+2*0.5),  #"spam"：有這個詞
  1-(2+0.5)/(2+2*0.5), #"ham"：沒有這個詞
  1-(1+0.5)/(2+2*0.5), #"rules"：沒有這個詞
  (1+0.5)/(2+2*0.5)   #"hello"：有這個詞
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

In [17]:
print('%.3f'%(model.predict(text)))
print('%.3f'%(p_if_spam/(p_if_spam+p_if_ham)))

6.000
0.835


In [18]:
# Should be about 0.83
assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

AssertionError: 